# Setup

## Installs

In [4]:
# Segmentation
# ! pip install --upgrade mxnet
# ! pip install torch==1.6.0+cpu torchvision==0.7.0+cpu -f https://download.pytorch.org/whl/torch_stable.html
# ! pip install --upgrade gluoncv

# Edge detection
# ! conda install scikit-image

# Depth estimation
# ! pip install -q transformers

## Imports

In [5]:
import numpy as np
import cv2
from os import listdir

In [6]:
# Import custom methods
import general_methods
import semantic_segmentation
import edge_detection
import depth_estimation
import geometry
import transforms

In [7]:
# Get pre-trained models
seg_model = semantic_segmentation.get_pretrained_model()

In [18]:
def pipeline(filename, wallpaper_filename):
    input_pil = general_methods.import_and_resize(filename)
    input_img = general_methods.import_mx_image(filename)

    # Segmentation
    mmask = semantic_segmentation.get_segementation(input_img, seg_model)
    labels = general_methods.get_labels_string(mmask)
    walls = general_methods.find_wall_indices(labels)
    other = general_methods.find_non_wall_indices(labels)
    segmented_input = semantic_segmentation.remove_walls(input_img, other)

    # Depth estimation
    depth_image = depth_estimation.estimate_depth(input_pil)

    # Edge detection
    input_cv2 = general_methods.import_cv2_image(filename)
    height = input_cv2.shape[0]
    width = input_cv2.shape[1]
    size = (width, height)
    edge_map = edge_detection.detect_edges(input_cv2)
    segmented_edges = edge_detection.get_segmented_edges(edge_map, walls)

    # Find corners
    hough_img = edge_detection.hough_transform(segmented_edges, height, width)
    vertical_lines = edge_detection.get_vertical_lines(hough_img)
    hough_colours = general_methods.get_labels_string(vertical_lines)
    hough_corners = edge_detection.get_hough_corners(hough_colours)
    mean_depth = depth_estimation.get_mean_depths(depth_image, other)
    matrix = general_methods.get_matrix(mean_depth)
    matrix_corners = depth_estimation.harris_corners(matrix)
    harris_colours = general_methods.get_labels_string(matrix_corners)
    _, harris_corners = np.where(harris_colours == "255,0,0")
    corner_inds = geometry.find_corners(hough_corners, harris_corners, width)
    only_walls = geometry.create_wall_corner_map(segmented_input, other, walls, corner_inds)

    # Find room geometry
    contours = geometry.find_contours(only_walls)
    corner_adj_geom = geometry.find_walls(contours, corner_inds)
    new_geom = geometry.find_quadrilaterals(corner_adj_geom, width)

    # Perspective transform
    wallpaper = general_methods.import_cv2_image(wallpaper_filename)
    result_1, result_2 = transforms.get_transformed_wallpaper(new_geom, height, width, size, wallpaper)

    # Create final image
    final_mask = transforms.get_wall_mask(new_geom, height, width, walls)
    final_output_1, final_output_2 = transforms.combine_wallpaper_and_input(input_cv2, final_mask, result_1, result_2, walls)
 
    return final_output_1, final_output_2
  

In [19]:
# Loop over all images in input folder
folder_dir = "images/inputs/rooms/test-set"
wallpaper_filename = "images/inputs/wallpaper/check-even.jpg"
 
for image in listdir(folder_dir):
    if image[0] >= 'a':
        filename = "images/inputs/rooms/test-set/" + image
        final_output_1, final_output_2 = pipeline(filename, wallpaper_filename) 
        cv2.imwrite("images/outputs/final-outputs/" + image, final_output_1)
        # cv2.imwrite("images/outputs/final-outputs/fake/simple-" + image, final_output_2)

c:\Users\Kim\Documents\Bath\Dissertation\wallpaper-previewer\depth_estimation.py:72: RuntimeWarning: Mean of empty slice
  mean_depth = np.nanmean(test_depth, axis=0)
c:\Users\Kim\Documents\Bath\Dissertation\wallpaper-previewer\depth_estimation.py:79: RuntimeWarning: Mean of empty slice
  mean_depth = np.nanmean(test_depth, axis=0)
c:\Users\Kim\Documents\Bath\Dissertation\wallpaper-previewer\depth_estimation.py:72: RuntimeWarning: Mean of empty slice
  mean_depth = np.nanmean(test_depth, axis=0)


<Figure size 640x480 with 0 Axes>

In [10]:
# filename = "images/inputs/rooms/test-set/dining-room3.jpg"
# wallpaper_filename = "images/inputs/wallpaper/check-even.jpg"
# final_output_1, final_output_2 = pipeline(filename, wallpaper_filename)
# cv2.imwrite("images/outputs/final-outputs/dining-room3.jpg", final_output_1)
# # cv2.imwrite("images/outputs/final-outputs/test-simple.png", final_output_2)